In [1]:
import numpy as np
import scipy.io as sio
import os
import glob
from scipy import sparse
import random as rd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

# from unittest import Testcase

Using TensorFlow backend.


# Reconstruct dataset

### create species dictinary:
    create a dictionary between species and their positions

In [4]:
import csv
import pandas as pd

data_path = '/home/hoangnh/PythonProject/RiceReconization/data/features-VIS/'
list_spec_feature_file_path = glob.glob(data_path + '*fullricespec.mat')
list_spatial_feature_file_path = glob.glob(data_path + '*Feat.mat')
list_spatial_feature_file_path.sort()
list_spec_feature_file_path.sort()
aboveLettersNum = 67
bellowLettersNum = 20

species = list()
firstSpecFilePath = list()
secondSpecFilePath = list()

firstSpatialFilePath = list()
secondSpatialFilePath = list()
for i in xrange (len(list_spec_feature_file_path)):
    key = list_spec_feature_file_path[i][aboveLettersNum:-bellowLettersNum]
    if i%2 ==0:
        species.append(key)
        firstSpatialFilePath.append(list_spatial_feature_file_path[i])
        firstSpecFilePath.append(list_spec_feature_file_path[i])
    else:
        secondSpatialFilePath.append(list_spatial_feature_file_path[i])
        secondSpecFilePath.append(list_spec_feature_file_path[i])
# list_spec_feature_file_path.sort()
# with open('dictionary.csv', 'w') as dictFile:
#     fieldNames = ['species', '1st_spatial_path', '2st_spatial_path', '1st_spec_path', '2st_spec_path']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
fieldNames = ['species', '1st_spatial_path', '2st_spatial_path', '1st_spec_path', '2st_spec_path']
records = np.asarray([species, firstSpatialFilePath, secondSpatialFilePath, firstSpecFilePath, secondSpecFilePath]).T
print records.shape
with open('dictionary.csv', 'wb') as dictFile:
#     fieldNames = ['species', '1st_spatial_path', '2st_spatial_path', '1st_spec_path', '2st_spec_path']
    writer = csv.writer(dictFile)
    writer.writerow(fieldNames)
    writer.writerows(records)
print 'end'

(90, 5)
end


In [3]:

n = np.random.randint(90)
print n
print species[n]
print firstSpatialFilePath[n]
print firstSpecFilePath[n]
print secondSpatialFilePath[n]
print secondSpecFilePath[n]

66
KN5
/home/hoangnh/PythonProject/RiceReconization/data/features-VIS/139_KN5-01_fullricespec.mat
/home/hoangnh/PythonProject/RiceReconization/data/features-VIS/139_KN5-01_fullricespec.mat
/home/hoangnh/PythonProject/RiceReconization/data/features-VIS/140_KN5-02_spatialFeat.mat
/home/hoangnh/PythonProject/RiceReconization/data/features-VIS/140_KN5-02_spatialFeat.mat


# load features and analyse them

## list all mat files on datapath

In [3]:
data_path = '/home/hoangnh/PythonProject/RiceReconization/data/features-VIS/'
list_spec_feature_file_path = glob.glob(data_path + '*fullricespec.mat')
list_spatial_feature_file_path = glob.glob(data_path + '*Feat.mat')

print len(list_spec_feature_file_path)
print len(list_spatial_feature_file_path)

list_spatial_feature_file_path.sort()
list_spec_feature_file_path.sort()

# for filePath in list_spatial_feature_file_path:
#     print filePath


180
180


In [4]:
# temp = glob.glob
print list_spatial_feature_file_path[1][67: -19]

BC15


### note: khong co giong thu 67, 82, 83, 84, 105, 106

### Read feature from file functions

In [5]:
def readSpecFeature(file_name):
    content = sio.loadmat(file_name)
    return content['fullspecData']

In [6]:
def readSpatiaFeature(file_name):
#     print file_name
    content = sio.loadmat(file_name)
    return content['spatialMat']

### load spatial features of a species
#### params:
    spPositions: postions of this species in list of species
    seedsNum: number of seeds
    featuresNum: number of features for one species
    list_file_path: file paths of all species
#### return:
    listFeature: features of all seeds of this species

In [7]:
seedsNumber = 96
featuresDimension = 6
speciesNumber = 6
def loadSpatialFeature(spPositions, seedsNum = 96, featuresNum = 6, list_file_path = list_spatial_feature_file_path):
    listFeature = list()
    for i in spPositions:
        listFeature.append(readSpatiaFeature(list_file_path[i]))
    listFeature = np.asarray(listFeature)
    listFeature = listFeature.reshape(seedsNum, featuresNum)
    return listFeature

### prepare spatial sub-dataset


In [8]:
species1Position = np.array([18,19])
species2Position = np.array([42,43])
species3Position = np.array([4,5])
species4Position = np.array([26,27])
species5Position = np.array([58,59])
species6Position = np.array([10,11])
sp1Features = loadSpatialFeature(species1Position)
sp2Features = loadSpatialFeature(species2Position)
sp3Features = loadSpatialFeature(species3Position)
sp4Features = loadSpatialFeature(species4Position)
sp5Features = loadSpatialFeature(species5Position)
sp6Features = loadSpatialFeature(species6Position)

### check features dementions

In [9]:
print sp1Features.shape
print sp2Features.shape
print sp3Features.shape
print sp4Features.shape
print sp5Features.shape
print sp6Features.shape

(96, 6)
(96, 6)
(96, 6)
(96, 6)
(96, 6)
(96, 6)


### separate function

In [10]:
def separate(features, trainRatio = 0.84):
    np.random.shuffle(features)
    trainingFeatures = features[0:int(trainRatio*features.shape[0])]
    testFeatures = features[int(trainRatio*features.shape[0]): features.shape[0]]
    return trainingFeatures, testFeatures

### test separate function

In [11]:
separate(np.asarray([1,2,3,4,5,6,7,8,9,10]), 0.84)

(array([ 1,  3,  8,  6,  2,  9,  4, 10]), array([7, 5]))

### Separate dataset

In [12]:
trainRatio = 0.84
sp1Training, sp1Test = separate(sp1Features, trainRatio)
sp2Training, sp2Test = separate(sp2Features, trainRatio)
sp3Training, sp3Test = separate(sp3Features, trainRatio)
sp4Training, sp4Test = separate(sp4Features, trainRatio)
sp5Training, sp5Test = separate(sp5Features, trainRatio)
sp6Training, sp6Test = separate(sp6Features, trainRatio)


### Assign labels for features

In [13]:
trainingData = np.concatenate((sp1Training, sp2Training, sp3Training, sp4Training, sp5Training, sp6Training), axis = 0)
testData = np.concatenate((sp1Test, sp2Test, sp3Test, sp4Test, sp5Test, sp6Test), axis = 0)
testNum = seedsNumber - int(trainRatio * seedsNumber)
# originLabels = np.asarray([])
for i in range(0, speciesNumber):
    if i == 0:
        trainingLabels = np.asarray([i] * int(trainRatio * seedsNumber)).T
        testLabels = np.asarray([i] * (seedsNumber - int(trainRatio * seedsNumber))).T
    else:
        trainingLabels = np.concatenate((trainingLabels, np.asarray([i] * int(trainRatio * seedsNumber)).T), axis = 0)
        testLabels = np.concatenate((testLabels, np.asarray([i] *  testNum).T), axis = 0)

# originLabels = np.asarray([0] * seedsNumber)

### Normalize dataset

In [14]:
standarVector = [100000, 1000, 100,  1, 0.1, 1]
trainingData1 = (trainingData/standarVector).T
testData1 = (testData/standarVector).T
# print (trainingData1[0:1, : ]).shape
# print (trainingData1[:, 0:3])

### soft max, one host coding, and loss function

In [15]:
def softmax(V):
    e_V = np.exp(V - np.max(V, axis = 0, keepdims = True))
    Z = e_V / e_V.sum(axis = 0)
    return Z

def oneHostDense(y, classNum = 6):
    Y = sparse.coo_matrix((
        np.ones_like(y), (y, np.arange(len(y)))), shape = (classNum, len(y))).toarray()
    return Y

def cost(Y, Yhat):
    return -np.sum(Y*np.log(Yhat))/Y.shape[1]

### Training

### Hand made network

In [ ]:
# d0 = featuresDimension 
# d1 = h = 6
# # d1 is the output number of  hiden layer 1
# # h is the inputnumer of input number of output layer
# d2 = C = speciesNumber 
# # d2 is the output number of input layer (layer 2)
# # C is the number of species

# # initialize parameters randomly
# W1 = 0.0001 * np.random.randn(d0, d1) 
# b1 = np.zeros((d1, 1))
# #weights, bias between input layer and hidden layer

# W2 = 0.0001 * np.random.randn(d1, d2) 
# b2 = np.zeros((d2, 1))
# #weights, bias between hidden layer 2 and output layer

# denseTrainingLabels = oneHostDense(trainingLabels, C)
# trainingSampleNum = trainingData1.shape[1]
# learningRate = 1

# iteration = 10000

# for i in xrange(iteration):
#     Z1 = np.dot(W1.T, trainingData1) + b1 
#     #Z1 is the input of first hidden layer
#     A1 = np.maximum(Z1, 0)
#     #A1 is the output of first hidden layer
#     Z2 = np.dot(W2.T, A1) + b2
#     trainingPredictions = softmax(Z2)
#     #backpropagation
#     E2 = (trainingPredictions - denseTrainingLabels)/ trainingSampleNum
#     dW2 = np.dot(A1, E2.T)
#     db2 = np.sum(E2, axis = 1, keepdims = True)
    
#     E1 = np.dot(W2, E2)
#     E1[Z1 <= 0] = 0 #gradient of ReLU
#     dW1 = np.dot(trainingData1, E1.T)
#     db1 = np.sum(E1, axis = 1, keepdims = True)
    
#     #Gradient Descent update
#     W1 += -learningRate * dW1
#     b1 += -learningRate * db1
#     W2 += -learningRate * dW2
#     b2 += -learningRate * db2
    
#     if i%1000 == 0:
#         loss = cost(denseTrainingLabels, trainingPredictions)
#         print ("iteration %d, loss: %f" %(i, loss))


### Using keras

In [21]:
# create model
denseTrainingLabels = oneHostDense(trainingLabels, speciesNumber)
model = Sequential()
model.add(Dense(12, input_dim=6, activation='relu'))
# model.add(Flatten(name='flatten'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(trainingData1.T, denseTrainingLabels.T, epochs=200, batch_size=10)
scores = model.evaluate(testData1.T, oneHostDense(testLabels).T)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/200
480/480 [==============================] - 0s - loss: 0.4512 - acc: 0.8333     
Epoch 2/200
480/480 [==============================] - 0s - loss: 0.4508 - acc: 0.8333     
Epoch 3/200
480/480 [==============================] - 0s - loss: 0.4506 - acc: 0.8333     
Epoch 4/200
480/480 [==============================] - 0s - loss: 0.4504 - acc: 0.8333     
Epoch 5/200
480/480 [==============================] - 0s - loss: 0.4503 - acc: 0.8333     
Epoch 6/200
480/480 [==============================] - 0s - loss: 0.4502 - acc: 0.8333     
Epoch 7/200
480/480 [==============================] - 0s - loss: 0.4499 - acc: 0.8333     
Epoch 8/200
480/480 [==============================] - 0s - loss: 0.4498 - acc: 0.8333     
Epoch 9/200
480/480 [==============================] - 0s - loss: 0.4495 - acc: 0.8333     
Epoch 10/200
480/480 [==============================] - 0s - loss: 0.4492 - acc: 0.8333     
Epoch 11/200
480/480 [==============================] - 0s - loss: 0.4487 - acc

480/480 [==============================] - 0s - loss: 0.2828 - acc: 0.8740     
Epoch 89/200
480/480 [==============================] - 0s - loss: 0.2801 - acc: 0.8757     
Epoch 90/200
480/480 [==============================] - 0s - loss: 0.2799 - acc: 0.8753     
Epoch 91/200
480/480 [==============================] - 0s - loss: 0.2771 - acc: 0.8771     
Epoch 92/200
480/480 [==============================] - 0s - loss: 0.2749 - acc: 0.8764     
Epoch 93/200
480/480 [==============================] - 0s - loss: 0.2731 - acc: 0.8771     
Epoch 94/200
480/480 [==============================] - 0s - loss: 0.2711 - acc: 0.8792     
Epoch 95/200
480/480 [==============================] - 0s - loss: 0.2697 - acc: 0.8771     
Epoch 96/200
480/480 [==============================] - 0s - loss: 0.2683 - acc: 0.8781     
Epoch 97/200
480/480 [==============================] - 0s - loss: 0.2667 - acc: 0.8781     
Epoch 98/200
480/480 [==============================] - 0s - loss: 0.2664 - acc: 0.

480/480 [==============================] - 0s - loss: 0.2032 - acc: 0.8972     
Epoch 176/200
480/480 [==============================] - 0s - loss: 0.2025 - acc: 0.8993     
Epoch 177/200
480/480 [==============================] - 0s - loss: 0.2025 - acc: 0.8972     
Epoch 178/200
480/480 [==============================] - 0s - loss: 0.2024 - acc: 0.8990     
Epoch 179/200
480/480 [==============================] - 0s - loss: 0.2031 - acc: 0.8965     
Epoch 180/200
480/480 [==============================] - 0s - loss: 0.2023 - acc: 0.8990     
Epoch 181/200
480/480 [==============================] - 0s - loss: 0.2016 - acc: 0.8993     
Epoch 182/200
480/480 [==============================] - 0s - loss: 0.2014 - acc: 0.8990     
Epoch 183/200
480/480 [==============================] - 0s - loss: 0.2001 - acc: 0.8990     
Epoch 184/200
480/480 [==============================] - 0s - loss: 0.2012 - acc: 0.9000     
Epoch 185/200
480/480 [==============================] - 0s - loss: 0.2014

In [ ]:
Z1 = np.dot(W1.T, testData1) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2
predicted_class = np.argmax(Z2, axis=0)
print('training accuracy: %.2f %%' % (100*np.mean(predicted_class == testLabels)))

In [ ]:
# temp =np.asarray([[3.79972124e+08, 3.56288960e+08, 3.82851206e+08, 4.07529309e+08 , 4.10980228e+08, 4.12379986e+08, 3.89384120e+08, 3.67045052e+08, 3.89184024e+08, 3.46503916e+08] ,[-1.81584610e+09, -1.70266680e+09, -1.82960492e+09, -1.94753893e+09 ,-1.96403051e+09, -1.97071980e+09, -1.86082502e+09, -1.75406900e+09 ,-1.85986879e+09, -1.65590512e+09] ,[3.89424382e+08, 3.65152072e+08, 3.92375085e+08, 4.17667085e+08, 4.21203850e+08, 4.22638428e+08, 3.99070513e+08, 3.76175734e+08, 3.98865440e+08, 3.55123613e+08] ,[3.72062517e+08, 3.48872349e+08, 3.74881667e+08, 3.99046063e+08, 4.02425148e+08, 4.03795767e+08, 3.81278590e+08, 3.59404538e+08, 3.81082660e+08, 3.39290993e+08], [3.77618022e+08, 3.54081587e+08, 3.80479267e+08, 4.05004478e+08, 4.08434017e+08, 4.09825103e+08, 3.86971707e+08, 3.64771039e+08, 3.86772851e+08, 3.44357166e+08], [2.96762487e+08, 2.78265671e+08, 2.99011082e+08, 3.18284957e+08, 3.20980164e+08, 3.22073390e+08, 3.04113361e+08, 2.86666299e+08, 3.03957084e+08, 2.70623441e+08]])
# # temp =np.asarray([[3,2,1], [5,6,7]])
# print (temp.shape)
# print (softmax(temp[:, 1]))
# a = softmax(temp)
# print (a)
# scores = [3.0e8, 1.0e8, 2e8]
scores = [3.0, 1.0, 2]
print(softmax(scores))

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
import math
import numpy as np
import matplotlib.pyplot as plt

N = 100 # number of points per class
d0 = 2 # dimensionality
C = 3 # number of classes
X = np.zeros((d0, N*C)) # data matrix (each row = single example)
y = np.zeros(N*C, dtype='uint8') # class labels

for j in xrange(C):
    ix = range(N*j,N*(j+1))
    r = np.linspace(0.0,1,N) # radius
    t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
    X[:,ix] = np.c_[r*np.sin(t), r*np.cos(t)].T
    y[ix] = j
print (X[1,:])
print ('y shape = ', y.shape)


In [ ]:
def softmax(V):
    e_V = np.exp(V - np.max(V, axis = 0, keepdims = True))
    Z = e_V / e_V.sum(axis = 0)
    return Z

## One-hot coding
from scipy import sparse
def convert_labels(y, C = 3):
    Y = sparse.coo_matrix((np.ones_like(y),
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y

# cost or loss function
def cost(Y, Yhat):
    return -np.sum(Y*np.log(Yhat))/Y.shape[1]

In [ ]:
d0 = 2
d1 = h = 100 # size of hidden layer
d2 = C = 3

# test

X = trainingData1[1:3, 0:160]
d2 = C = 2
y = trainingLabels[0:160]
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))


Y = convert_labels(y, C)
N = X.shape[1]
eta = 1 # learning rate
for i in xrange(100):
    ## Feedforward
    Z1 = np.dot(W1.T, X) + b1
    A1 = np.maximum(Z1, 0)
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = softmax(Z2)
    
    # backpropagation
    E2 = (Yhat - Y )/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1[Z1 <= 0] = 0 # gradient of ReLU
    dW1 = np.dot(X, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)

    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

    # print loss after each 1000 iterations
    if i %1 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(Y, Yhat)
        print("iter %d, loss: %f, " %(i, loss))
#         print (dW1[1, 1])


In [ ]:
print (trainingData1.shape)
print (trainingData1[:, 7])
print (testLabels.shape)
# print (trainingLabels)
# print (testLabels)

In [ ]:
trainingData1 = trainingData.T
print (trainingData1[:, 1])
print (trainingData1[:, 90])
# print (1/0)